In [1]:
import pandas as pd
import numpy as np
pd.options.mode.chained_assignment = None  # default='warn'

load data

In [174]:
merged_df = pd.read_csv('data/main/data_set.csv')

In [ ]:
list(merged_df.columns)

rename for ols formular

In [175]:
merged_df = merged_df.rename(columns={'debt/equity_ratio_2019' : 'debtxtoxequity_ratio_2019', 
                                        'debt/asset_ratio_2019' : 'debtxtoxassest_ratio_2019',
                                        'debt/equity_ratio_2020' : 'debtxtoxequity_ratio_2020', 
                                        'debt/asset_ratio_2020' : 'debtxtoxassest_ratio_2020',
                                        'debt/equity_ratio_2021' : 'debtxtoxequity_ratio_2021', 
                                        'debt/asset_ratio_2021' : 'debtxtoxassest_ratio_2021',
                                        })

In [176]:
ratios = ['cash_ratio','quick_ratio','current_ratio','debtxtoxequity_ratio','equity_ratio','debtxtoxassest_ratio']
types = ['aid_grant','aid_loan']
results_dict = {}

In [259]:
#loop over year
for year in [2020,2021]:

    for id, type in enumerate(types, start=1):
        type_inverse_year = types[id-2] + '_' + str(year)
        type_year = type + '_' + str(year)
        data_unconfounded = merged_df[merged_df[type_inverse_year].isnull()]

        #loop over ratio
        if year == 2021: # Remove companies that received other aid due to possible confounding
                data_unconfounded = data_unconfounded[data_unconfounded[('aid_grant'+'_'+ str(year-1))].isnull()] ###
                data_unconfounded = data_unconfounded[data_unconfounded[('aid_loan'+'_'+ str(year-1))].isnull()] ###
        
        results = {}
        

        for ratio in ratios:
            

            ratio_year_prior = ratio + '_' + str(year-1)
            ratio_year = ratio + '_' + str(year)

            data_dd = data_unconfounded[[ratio_year_prior , ratio_year, type_year]]
            data_dd['treatment_group'] = np.where(data_dd[type_year].isna() , 0, 1)
            data_dd = data_dd.drop(type_year, axis=1)
            data_dd.replace([np.inf, -np.inf], np.nan, inplace=True)
            data_dd = data_dd.dropna()

            # data before the treatment
            df_before = data_dd[[ratio_year_prior, 'treatment_group']]
            df_before['t'] = 0
            df_before.columns = [ratio, 'treatment_group', 'time']

            # data after the treatment
            df_after = data_dd[[ratio_year, 'treatment_group']]
            df_after['t'] = 1
            df_after.columns = [ratio, 'treatment_group', 'time']

            # data for regression
            df_reg = pd.concat([df_before, df_after])

            # create the interaction 
            df_reg['interaction'] = df_reg.treatment_group * df_reg.time

            # drop extrem outliers
            df_reg = df_reg[df_reg[ratio] > 0 ]
            df_reg = df_reg.drop(df_reg[df_reg[ratio] > df_reg[ratio].quantile(.95)].index)
            

            from statsmodels.formula.api import ols

            formular = ratio + ' ~ treatment_group + time + interaction' 
            ols = ols(formular, data=df_reg).fit()

            cov = ols.params['interaction'].round(4)
            pv =  ols.pvalues['interaction'].round(3)
            
            
            if pv > 0.1:
                cov = str(cov)+'  '
            elif pv < 0.01:
                cov = str(cov)+'***'
            elif pv < 0.05:
                cov = str(cov)+'** '  
            elif pv < 0.01:
                 cov = str(cov)+'*  '
            elif pv > 0.1:
                 cov = str(cov)+'  '

            pv = ("%.3f" % pv)           
            pv = "("+str(pv)+")"

            ratio = ratio.replace("_", " ").replace("x", " ")
            results[ratio] = ' '.join((str(cov),pv))

        results_dict[type_year] = results

format output

In [257]:
df_results = pd.DataFrame.from_dict(results_dict, orient='index')
df_results.index.name='aid_ratio_year'
df_results.reset_index(inplace=True)
new_col = df_results['aid_ratio_year'].str.split('_',expand=True)
df_results['ratio'] = new_col[1]
df_results['year'] = new_col[2]
df_results.drop(columns =["aid_ratio_year"], inplace = True)
df_results.set_index(['year', 'ratio'],inplace=True)
df_results

cash ratio        quick ratio      current ratio  \
year ratio                                                            
2020 grant   -0.006   (0.521)  -0.0871   (0.457)  -0.1636   (0.204)   
     loan    0.065*** (0.000)  0.0905**  (0.029)  0.1518*** (0.001)   
2021 grant   0.093*** (0.000)   0.0357   (0.853)  -0.0279   (0.895)   
     loan   0.0388*** (0.000)     0.2039 (0.094)     0.2657 (0.072)   

           debt to equity ratio        equity ratio debt to assest ratio  
year ratio                                                                
2020 grant    0.5661**  (0.041)   -0.0095   (0.497)    0.0366**  (0.024)  
     loan     1.0924*** (0.000)  -0.0575*** (0.000)    0.0764*** (0.000)  
2021 grant    -0.3862   (0.126)    0.0136   (0.405)      -0.0319 (0.083)  
     loan        0.7023 (0.070)  -0.0453*** (0.002)    0.0558*** (0.002)

save data frame to latex

In [258]:
with open('/Users/marco/Library/CloudStorage/OneDrive-HertieSchool/Thesis/GitHub/thesis/paper/Tables/table_did.tex', 'w') as tf:
     tf.write(df_results.transpose().stack().to_latex(index=True, bold_rows=True, column_format='llrr'))

/var/folders/nr/94zdnpbn2xj3pwc_d9lfk0cw0000gn/T/ipykernel_48019/2811550283.py:2: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  tf.write(df_results.transpose().stack().to_latex(index=True, bold_rows=True, column_format='llrr'))
